<a href="https://colab.research.google.com/github/Azizkhaled/NLP-Concepts-and-Transformers/blob/main/Projects/TrainingPretrainedBert/MLM_NSP_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLM and NSP

In this notebook, we'll go through traing a BERT model for both masking predictiona and predicting if the setencet is a next sentence or not

### Initailize tokenizer and model

In [1]:
pip install transformers accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.7 MB/s eta 0:00:00


In [2]:
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')



### Load data, clean, prepare for NSP, and Tokenize

We will utilize the UN report titled "The Question of Palestine" for this project. The data can be accessed using the following link:

https://raw.githubusercontent.com/Azizkhaled/NLP/main/Data/UN_text.txt

We'll start by loading the data from the provided URL and removing any duplicate entries while preserving the order of appearance.

In [3]:
import requests

# function to remove duplicates while keeping the same order
def remove_duplicates_keep_order(input_list):
    seen = set()
    result = []

    for item in input_list:
        if item.strip() != '' and item not in seen:  # Check if the line is not empty and not seen before
            result.append(item)
            seen.add(item)

    return result


In [4]:
data = requests.get('https://raw.githubusercontent.com/Azizkhaled/NLP/main/Data/UN_text.txt')
text = data.text.split('\n')

text = remove_duplicates_keep_order(text)
print( len(text),': ', text[0:3])

886 :  ['The question of Palestine was brought before the United Nations shortly after the end of the Second World War.\r', 'The origins of the Palestine problem as an international issue, however, lie in events occurring towards the end of the First World War. These events led to a League of Nations decision to place Palestine under the administration of Great Britain as the Mandatory Power under the Mandates System adopted by the League. In principle, the Mandate was meant to be in the nature of a transitory phase until Palestine attained the status of a fully independent nation, a status provisionally recognized in the League’s Covenant, but in fact the Mandate’s historical evolution did not result in the emergence of Palestine as an independent nation.\r', 'The decision on the Mandate did not take into account the wishes of the people of Palestine, despite the Covenant’s requirements that “the wishes of these communities must be a principal consideration in the selection of the Man

We'll assign a 50% probability of using the genuine next sentence, and 50% probability of using another random sentence.

In [9]:
bag = [item for sentence in text for item in sentence.split('.') if item != '\r' and len(item)>10]  # remove empty and very short sentecnes
bag_size = len(bag)

# this bag will be used to pull from later on

In [10]:
bag_size

1561

#### Create our 50/50 NSP training data.

In [12]:
import random

sentence_a = []
sentence_b = []
label = []

for paragraph in text:
    sentences = [
        sentence for sentence in paragraph.split('.') if sentence != '\r' and len(sentence)>20
    ]
    num_sentences = len(sentences)
    if num_sentences > 1:
        start = random.randint(0, num_sentences-2)
        # 50/50 whether is IsNextSentence or NotNextSentence
        if random.random() >= 0.5:
            # this is IsNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(sentences[start+1])
            label.append(0)
        else:
            index = random.randint(0, bag_size-1)
            # this is NotNextSentence
            sentence_a.append(sentences[start])
            sentence_b.append(bag[index])
            label.append(1)

#### Tokenize the data

In [13]:
inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs.keys()


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

### Labelling for MLM and NSP
We can now create our labels tensor for MLM, and the next_sentence_label for NSP.

In [14]:
inputs['next_sentence_label'] = torch.LongTensor([label]).T


And the labels tensor is simply a clone of the input_ids tensor before masking.


In [15]:
inputs['labels'] = inputs.input_ids.detach().clone()

#### Randomly mask tokens

In [16]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# make sure we don't mask the CLS (101) and SEP tokens (102) and paddings (0)
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

In [17]:
## indices to be masked in each paragraph
selection = []

for i in range(inputs.input_ids.shape[0]):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist()
    )


In [18]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103


In [20]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'next_sentence_label', 'labels'])

## Training Method 1: Pytorch Training

###  Create a PyTorch dataset from our data

In [21]:
class UnDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [22]:
dataset = UnDataset(inputs)

### Initialize loader

In [23]:
loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [25]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### Start the training loop

In [26]:
from tqdm import tqdm  # for our progress bar

epochs = 3

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        next_sentence_label = batch['next_sentence_label'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/41 [00:00<?, ?it/s]<ipython-input-21-b95cca338fe7>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 2: 100%|██████████| 41/41 [00:37<00:00,  1.10it/s, loss=0.162]


## Training Method 2: HuggingFace Trainer function

In [27]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir='out',
    per_device_train_batch_size=8,
    num_train_epochs=3
)

In [28]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset
)

In [29]:
torch.cuda.empty_cache()

In [30]:
trainer.train()


<ipython-input-21-b95cca338fe7>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=123, training_loss=0.09452619009870823, metrics={'train_runtime': 111.8054, 'train_samples_per_second': 8.774, 'train_steps_per_second': 1.1, 'total_flos': 259988383272960.0, 'train_loss': 0.09452619009870823, 'epoch': 3.0})